In [161]:
!kaggle competitions download -c digit-recognizer -p .

digit-recognizer.zip: Skipping, found more recently modified local copy (use --force to force download)


In [162]:
import pandas as pd
import numpy as np
import torch.optim as optim
import torch.nn.functional as F
import zipfile
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.decomposition import PCA

from sklearn.model_selection import train_test_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [163]:
with zipfile.ZipFile("digit-recognizer.zip", "r") as zip_ref:
    zip_ref.extractall(".")  


In [164]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [165]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [166]:
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [167]:
X = train.drop("label", axis=1).values / 255.0  # Normalize
y = train["label"].values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
pca = PCA(n_components=300)  # Reduce 784 → 300 features
X_train_pca = pca.fit_transform(X_train)
X_val_pca=pca.transform(X_val)
X_test_pca = pca.transform(test.values / 255.0)


In [168]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
X_test_tensor = torch.tensor(test.values / 255.0, dtype=torch.float32)

# X_train_tensor = torch.tensor(X_train_pca, dtype=torch.float32)
# y_train_tensor = torch.tensor(y_train, dtype=torch.long)
# X_val_tensor = torch.tensor(X_val_pca, dtype=torch.float32)
# y_val_tensor = torch.tensor(y_val, dtype=torch.long)
# X_test_tensor = torch.tensor(X_test_pca, dtype=torch.float32)

batch_size = 128
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=batch_size, shuffle=False)
test_loader = DataLoader(TensorDataset(X_test_tensor), batch_size=batch_size, shuffle=False)



In [169]:
def swish(x):
    return x * torch.sigmoid(x)

# class MLP(nn.Module):
#     def __init__(self, input_size=784,hidden_size=512,output_size=10,dropout_rate=0.3):
#         super(MLP, self).__init__()
#         self.fc1 = nn.Linear(input_size, hidden_size)
#         self.bn1 = nn.BatchNorm1d(hidden_size)
#         self.dropout1 = nn.Dropout(dropout_rate)

#         self.fc2 = nn.Linear(hidden_size, hidden_size)
#         self.bn2 = nn.BatchNorm1d(hidden_size)
#         self.dropout2 = nn.Dropout(dropout_rate)

#         self.fc3 = nn.Linear(hidden_size, hidden_size)
#         self.bn3 = nn.BatchNorm1d(hidden_size)
#         self.dropout3 = nn.Dropout(dropout_rate)

#         self.fc4 = nn.Linear(hidden_size, hidden_size)
#         self.bn4 = nn.BatchNorm1d(hidden_size)
#         self.dropout4=nn.Dropout(dropout_rate)

#         self.fc5 = nn.Linear(hidden_size, hidden_size)
#         self.bn5 = nn.BatchNorm1d(hidden_size)
#         self.dropout5=nn.Dropout(dropout_rate)


#         self.fc_out = nn.Linear(hidden_size, output_size)

#     def forward(self, x):
#         x1 = torch.relu(self.bn1(self.fc1(x)))
#         x1 = self.dropout1(x1)
        
#         x2 = torch.relu(self.bn2(self.fc2(x1)))
#         x2 = self.dropout2(x2)
#         x2 = x1 + x2  # Skip connection
        
#         x3 = torch.relu(self.bn3(self.fc3(x2)))
#         x3 = self.dropout3(x3)
#         x3 = x2 + x3  # Skip connection

#         x4 = torch.relu(self.bn4(self.fc4(x3)))
#         x4 = self.dropout4(x4)
#         x4 = x3 + x4  # Skip connection

#         x5 = torch.relu(self.bn5(self.fc5(x4)))
#         x5 = self.dropout4(x5)
#         x5 = x4 + x5  # Skip connection

#         # Output layer (Softmax applied later in loss function)
#         out = self.fc_out(x5)
#         return out
class MLP(nn.Module):
    def __init__(self, input_size=784, hidden_size=1024, output_size=10, dropout_rate=0.3):
        super(MLP, self).__init__()
        
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(dropout_rate)
        
        self.hidden_layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList()
        self.dropouts = nn.ModuleList()
        
        for _ in range(9):  # 9 additional hidden layers
            self.hidden_layers.append(nn.Linear(hidden_size, hidden_size))
            self.batch_norms.append(nn.BatchNorm1d(hidden_size))
            self.dropouts.append(nn.Dropout(dropout_rate))
        
        self.fc_out = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x_res = swish(self.bn1(self.fc1(x)))
        x_res = self.dropout1(x_res)
        
        for i in range(9):
            x_new = swish(self.batch_norms[i](self.hidden_layers[i](x_res)))
            x_new = self.dropouts[i](x_new)
            x_res = x_res + x_new  # Skip connection
        
        out = self.fc_out(x_res)
        return out



In [171]:
model = MLP(hidden_size=512).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.7)   #reduce to 50% every 10 steps

epochs=100
for epoch in range(epochs):
    model.train()
    total_loss, correct = 0, 0
    
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        optimizer.zero_grad()
        outputs = model(X_batch)
        
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        correct += (outputs.argmax(1) == y_batch).sum().item()

    train_acc = correct / len(X_train)
    
    # Validation
    model.eval()
    correct = 0
    with torch.no_grad():
        for X_val_batch, y_val_batch in val_loader:
            X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)
            outputs = model(X_val_batch)
            correct += (outputs.argmax(1) == y_val_batch).sum().item()
    
    val_acc = correct / len(X_val)
    
    print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss:.4f} - Train Acc: {train_acc:.4f} - Val Acc: {val_acc:.4f}")

    # Adjust learning rate
    scheduler.step()



Epoch 1/100 - Loss: 116.0113 - Train Acc: 0.8833 - Val Acc: 0.9475
Epoch 2/100 - Loss: 51.6644 - Train Acc: 0.9438 - Val Acc: 0.9625
Epoch 3/100 - Loss: 38.9105 - Train Acc: 0.9559 - Val Acc: 0.9627
Epoch 4/100 - Loss: 32.5613 - Train Acc: 0.9615 - Val Acc: 0.9671
Epoch 5/100 - Loss: 26.9018 - Train Acc: 0.9681 - Val Acc: 0.9677
Epoch 6/100 - Loss: 25.2647 - Train Acc: 0.9705 - Val Acc: 0.9730
Epoch 7/100 - Loss: 27.6980 - Train Acc: 0.9695 - Val Acc: 0.9742
Epoch 8/100 - Loss: 20.4381 - Train Acc: 0.9759 - Val Acc: 0.9749
Epoch 9/100 - Loss: 20.0132 - Train Acc: 0.9770 - Val Acc: 0.9723
Epoch 10/100 - Loss: 20.4631 - Train Acc: 0.9765 - Val Acc: 0.9744
Epoch 11/100 - Loss: 14.8030 - Train Acc: 0.9807 - Val Acc: 0.9767
Epoch 12/100 - Loss: 12.8337 - Train Acc: 0.9839 - Val Acc: 0.9774
Epoch 13/100 - Loss: 11.4847 - Train Acc: 0.9854 - Val Acc: 0.9780
Epoch 14/100 - Loss: 10.6512 - Train Acc: 0.9868 - Val Acc: 0.9796
Epoch 15/100 - Loss: 9.6175 - Train Acc: 0.9872 - Val Acc: 0.9775
Epoc

In [ ]:
model.eval()
predictions = []
with torch.no_grad():
    for X_test_batch in test_loader:
        X_test_batch = X_test_batch[0].to(device)
        outputs = model(X_test_batch)
        predictions.extend(outputs.argmax(1).cpu().numpy())

# Create submission file
submission = pd.DataFrame({"ImageId": np.arange(1, len(predictions) + 1), "Label": predictions})
submission.to_csv("submission.csv", index=False)
